In [3]:
%pip install dash-auth
%pip install dash-bootstrap-components

  Using cached dash_auth-2.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached dash-3.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached plotly-6.5.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached retrying-1.4.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached markupsafe-3.0.3-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
  Usi


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached dash_bootstrap_components-2.0.4-py3-none-any.whl.metadata (18 kB)
Using cached dash_bootstrap_components-2.0.4-py3-none-any.whl (204 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
"""
DASHBOARD BEPENSA - VERSION COMPLETA
Sidebar ancho estilo Bootstrap United
Home con tarjetas clickeables
Multipágina: Inicio, Panel, Ingresos, Unidades, Demanda, Rutas
Dark mode completo
"""

import dash
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import base64
import plotly.express as px
import datetime as dt

import pickle
import pandas as pd
import plotly.express as px
from dash import html, dcc
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc



# ============================
# Cargar modelo y preprocesamiento
# ============================
with open('modelo_rendimiento_xgbg.pickle', 'rb') as f_model:
    pipe_xgbg = pickle.load(f_model)

with open('prepro_rendimiento.pickle', 'rb') as f_prepro:
    prepro_info = pickle.load(f_prepro)

scaler = prepro_info['scaler']
num_cols = prepro_info['num_cols']
viaje_dummy_cols = prepro_info['viaje_dummy_cols']

viajes_disponibles = sorted({
    col.replace('Viaje_', '') for col in viaje_dummy_cols
})

rend_promedio_ruta = {
    "LM/ARR":   2.076040,
    "LM/MEX":   2.217559,
    "LM/MEXII": 2.251395,
    "LM/MIDCG": 2.091800,
    "LM/MTY":   1.747674,
    "LM/QRO":   1.974058,
}

def predecir_rendimiento(distancia, porc_ralenti, vel_max, peso_kgs, viaje):
    """Arma el vector de features y devuelve el rendimiento predicho."""
    X_num_new = pd.DataFrame([{
        'Distancia': distancia,
        'Porcentaje de tiempo en ralentí': porc_ralenti,
        'Velocidad máxima': vel_max,
        'Peso Kgs': peso_kgs,
    }])

    # Orden correcto
    X_num_new = X_num_new[num_cols]

    # Escalar numéricas
    X_num_scaled = pd.DataFrame(
        scaler.transform(X_num_new),
        columns=num_cols
    )

    # Dummies de viaje
    dummy_dict = {col: 0 for col in viaje_dummy_cols}
    col_name = f'Viaje_{viaje}'

    if col_name not in dummy_dict:
        raise ValueError(
            f"El viaje '{viaje}' no está en las columnas dummy del modelo. "
            f"Usa uno de: {viajes_disponibles}"
        )

    dummy_dict[col_name] = 1
    X_cat_new = pd.DataFrame([dummy_dict])

    # Concatenar numéricas + categóricas
    X_new = pd.concat([X_num_scaled, X_cat_new], axis=1)

    # Predicción
    y_pred_new = pipe_xgbg.predict(X_new)[0]
    return y_pred_new

# -------------------------------------------------------------------
#                       CONFIGURACIÓN GENERAL
# -------------------------------------------------------------------

# Cargar logo
image_filename = "bepensa_logo.png"
encoded_image = base64.b64encode(open(image_filename, "rb").read()).decode()

# Tema general
app = Dash(
    __name__,
    suppress_callback_exceptions=True,
    external_stylesheets=[dbc.themes.LUX, dbc.icons.BOOTSTRAP],
)
app.title = "Bepensa Logistics Dashboard"

COLOR_ORANGE = "#f57c3b"

# Datos de ejemplo
df = px.data.gapminder().query("year == 2007")
df2 = px.data.stocks()

# -------------------------------------------------------------------
#                              SIDEBAR
# -------------------------------------------------------------------

def sidebar():
    return html.Div(
        [
            # LOGO PRINCIPAL
            html.Img(
                src=f"data:image/png;base64,{encoded_image}",
                style={
                    "height": "60px",
                    "margin-bottom": "25px",
                    "object-fit": "contain",
                    "margin-left": "10px"
                },
            ),

            # NAV CON ICONOS
            dbc.Nav(
                [
                    dbc.NavLink(
                        html.Img(src="/assets/icon_home.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/",
                        active="exact",
                        className="sidebar-icon-link",
                    ),

                    dbc.NavLink(
                        html.Img(src="/assets/icon_panel.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/panel",
                        active="exact",
                        className="sidebar-icon-link",
                    ),

                    dbc.NavLink(
                        html.Img(src="/assets/icon_ingresos.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/ingresos",
                        active="exact",
                        className="sidebar-icon-link",
                    ),

                    dbc.NavLink(
                        html.Img(src="/assets/icon_unidades.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/unidades",
                        active="exact",
                        className="sidebar-icon-link",
                    ),

                    dbc.NavLink(
                        html.Img(src="/assets/icon_demanda.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/demanda",
                        active="exact",
                        className="sidebar-icon-link",
                    ),

                    dbc.NavLink(
                        html.Img(src="/assets/icon_rutas.png",
                                style={"height": "32px", "marginBottom": "14px"}),
                        href="/rutas",
                        active="exact",
                        className="sidebar-icon-link",
                    ),
                ],
                vertical=True,
                pills=True,
                style={"marginTop": "5px"},
            ),

            html.Hr(),

            # DARK MODE SWITCH
            html.Div(
                [
                    html.Span("Modo oscuro", style={"fontSize": "13px"}),
                    dbc.Switch(id="theme-switch", value=False, class_name="ms-2"),
                ],
                style={"marginTop": "20px"},
            ),
        ],

        id="sidebar",
        style={
            "position": "fixed",
            "top": 0,
            "left": 0,
            "bottom": 0,
            "width": "8rem",
            "padding": "1rem",
            "background-color": "#ffffff",
            "overflow-x": "hidden",
            "transition": "all 0.3s",
            "box-shadow": "2px 0 6px rgba(0,0,0,0.1)",
            "textAlign": "center",
        },
    )

# -------------------------------------------------------
#                     Simulador
# --------------------------------------------------------
def prediction_rendimiento_card():
    return dbc.Card(
        dbc.CardBody(
            [
                html.H5("Predicción de rendimiento", className="card-title"),
                html.P(
                    "Ingresa los datos del viaje para estimar el rendimiento esperado (km/l).",
                    className="card-text"
                ),

                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dbc.Label("Distancia (km)"),
                                dbc.Input(
                                    id="input-distancia-rutas",
                                    type="number",
                                    min=0,
                                    step=0.1,
                                    value=0.0,
                                ),
                            ],
                            md=6,
                        ),
                        dbc.Col(
                            [
                                dbc.Label("Porcentaje de tiempo en ralentí (%)"),
                                dbc.Input(
                                    id="input-ralenti-rutas",
                                    type="number",
                                    min=0,
                                    max=100,
                                    step=0.1,
                                    value=0.0,
                                ),
                            ],
                            md=6,
                        ),
                    ],
                    className="g-2",
                ),

                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dbc.Label("Velocidad máxima (km/h)"),
                                dbc.Input(
                                    id="input-vel-max-rutas",
                                    type="number",
                                    min=0,
                                    step=0.1,
                                    value=0.0,
                                ),
                            ],
                            md=6,
                        ),
                        dbc.Col(
                            [
                                dbc.Label("Peso (kg)"),
                                dbc.Input(
                                    id="input-peso-rutas",
                                    type="number",
                                    min=0,
                                    step=1,
                                    value=0.0,
                                ),
                            ],
                            md=6,
                        ),
                    ],
                    className="g-2",
                    style={"marginTop": "8px"},
                ),

                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dbc.Label("Viaje"),
                                dcc.Dropdown(
                                    id="input-viaje-rutas",
                                    options=[
                                        {"label": v, "value": v}
                                        for v in viajes_disponibles
                                    ],
                                    value=(
                                        viajes_disponibles[0]
                                        if viajes_disponibles
                                        else None
                                    ),
                                    clearable=False,
                                    className="dropdown-dark",  # para modo oscuro
                                ),
                            ],
                            md=12,
                        ),
                    ],
                    style={"marginTop": "8px"},
                ),

                dbc.Button(
                    "Calcular rendimiento esperado",
                    id="predict-button-rutas",
                    color="primary",
                    className="mt-3",
                ),

                html.Hr(),

                html.Div(
                    id="output-prediccion-rutas",
                    className="mt-2 fw-bold",
                    style={"fontSize": "18px"},
                ),

                html.Div(           # mensaje de riesgo
                    id="output-riesgo-rutas",
                    className="mt-1",
                    style={"fontSize": "14px"},
                ),

                html.Div(
                    id="output-error-rutas",
                    className="mt-1 text-danger",
                ),
            ]
        )
    )

# -------------------------------------------------------------------
#                 COMPONENTES REUTILIZABLES
# -------------------------------------------------------------------

def page_header(title, subtitle="Bepensa Logistics"):
    return dbc.Row(
        [
            dbc.Col(
                [
                    html.H2(title),
                    html.H6(subtitle),
                ]
            ),
            dbc.Col(
                html.Div(id="clock", className="clock-box"),
                width=3,
            ),
        ],
        style={"marginBottom": "20px"},
        align="center",
    )

def section_card(icon, title, subtitle, href):
    return dcc.Link(
        dbc.Card(
            dbc.CardBody(
                [
                    html.I(className=f"{icon}", style={"fontSize": "28px", "color": COLOR_ORANGE}),
                    html.H4(title, style={"marginTop": "10px"}),
                    html.P(subtitle, style={"fontSize": "13px", "color": "#555"}),
                ]
            ),
            style={
                "borderRadius": "16px",
                "border": "none",
                "borderTop": f"6px solid {COLOR_ORANGE}",
                "boxShadow": "0 4px 12px rgba(0,0,0,0.15)",
                "height": "170px",
                "transition": "0.2s",
                "textAlign": "center",
            },
            className="mb-4 section-card",
        ),
        href=href,
        style={"textDecoration": "none", "color": "black"},
    )


# -------------------------------------------------------------------
#                      LAYOUTS DE CADA PÁGINA
# -------------------------------------------------------------------

def layout_inicio():
    return html.Div(
        [
            page_header("Dashboard de Control Logístico"),

            html.H4("Secciones", style={"marginBottom": "20px"}),

            dbc.Row(
                [
                    dbc.Col(section_card("bi bi-cash-coin", "Ingresos",
                                        "Detalle de ingresos y costos", "/ingresos"), md=4),
                    dbc.Col(section_card("bi bi-truck", "Unidades",
                                        "Flota y operación", "/unidades"), md=4),
                    dbc.Col(section_card("bi bi-graph-up-arrow", "Demanda",
                                        "Comportamiento de la demanda", "/demanda"), md=4),
                ]
            ),

            dbc.Row(
                [
                    dbc.Col(section_card("bi bi-signpost-split", "Rutas",
                                        "Corredores logísticos", "/rutas"), md=4),
                    dbc.Col(section_card("bi bi-layout-text-sidebar", "Panel de Control",
                                        "Resumen ejecutivo", "/panel"), md=4),
                ],
                style={"marginTop": "10px"},
            ),
        ],
        style={"padding": "20px"},
    )


def layout_panel():
    fig1 = px.bar(df, x="country", y="gdpPercap", title="Meta Mensual (dummy)")
    fig2 = px.line(df2, x="date", y=["GOOG"], title="Demanda (dummy)")

    return html.Div(
        [
            page_header("Panel de Control"),
            dbc.Row(
                [
                    dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(figure=fig1))), md=6),
                    dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(figure=fig2))), md=6),
                ]
            ),
        ],
        style={"padding": "20px"},
    )


def layout_ingresos():
    fig = px.histogram(df, x="gdpPercap", nbins=10, title="Ingresos dummy")
    return html.Div(
        [
            page_header("Ingresos"),
            dbc.Row([dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(figure=fig))), md=12)])
        ],
        style={"padding": "20px"},
    )


def layout_unidades():
    fig = px.scatter(df, x="gdpPercap", y="lifeExp", color="continent")
    return html.Div(
        [
            page_header("Unidades"),
            dbc.Row([dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(figure=fig))), md=12)])
        ],
        style={"padding": "20px"},
    )


def layout_demanda():
    fig = px.line(df2, x="date", y="AAPL", title="Demanda dummy")
    return html.Div(
        [
            page_header("Demanda"),
            dbc.Row([dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(figure=fig))), md=12)])
        ],
        style={"padding": "20px"},
    )


def layout_rutas():
    fig = px.choropleth(df, locations="iso_alpha", color="continent", title="Mapa dummy")

    return html.Div(
        [
            page_header("Rutas"),
            dbc.Row(
                [
                    dbc.Col(prediction_rendimiento_card(), md=4),
                    dbc.Col(
                        dbc.Card(
                            dbc.CardBody(
                                dcc.Graph(figure=fig, id="mapa-rutas")
                            )
                        ),
                        md=8,
                    ),
                ],
                className="mt-3",
            ),
        ],
        style={"padding": "20px"},
    )

# -------------------------------------------------------------------
#                          LAYOUT GENERAL
# -------------------------------------------------------------------

# -------------------------------------------------------------------
#                          LAYOUT GENERAL
# -------------------------------------------------------------------

app.layout = html.Div(
    [
        dcc.Location(id="url"),
        dcc.Interval(id="clock-interval", interval=1000, n_intervals=0),

        sidebar(),

        html.Div(
            id="page-content",
            style={
                "marginLeft": "8rem",
                "padding": "2rem",
                "transition": "0.3s",
            },
        ),
    ],
    id="app-root",
    className="light-theme",

    # Imagen de fondo desde assets
    style={
        "backgroundImage": "url('/assets/bepensa_logos.png')",
        "backgroundPosition": "top right",
        "backgroundSize": "1300px",
        "backgroundRepeat": "no-repeat",
        "backgroundAttachment": "fixed"
    }
)


# -------------------------------------------------------------------
#                             CALLBACKS
# -------------------------------------------------------------------
@app.callback(
    Output("output-prediccion-rutas", "children"),
    Output("output-riesgo-rutas", "children"),
    Output("output-error-rutas", "children"),
    Input("predict-button-rutas", "n_clicks"),
    State("input-distancia-rutas", "value"),
    State("input-ralenti-rutas", "value"),
    State("input-vel-max-rutas", "value"),
    State("input-peso-rutas", "value"),
    State("input-viaje-rutas", "value"),
)
def actualizar_prediccion_rutas(
    n_clicks, distancia, porc_ralenti, vel_max, peso_kgs, viaje
):
    if not n_clicks:
        return "", "", ""

    # Validación básica
    try:
        distancia = float(distancia or 0)
        porc_ralenti = float(porc_ralenti or 0)
        vel_max = float(vel_max or 0)
        peso_kgs = float(peso_kgs or 0)
    except ValueError:
        return "", "", "Error: verifica que todos los valores numéricos sean válidos."

    if viaje is None:
        return "", "", "Error: selecciona un viaje."

    try:
        pred = predecir_rendimiento(
            distancia=distancia,
            porc_ralenti=porc_ralenti,
            vel_max=vel_max,
            peso_kgs=peso_kgs,
            viaje=viaje,
        )
        texto_pred = f"El modelo predice un rendimiento de: {pred:.2f} km/l"

        # -------- Lógica de riesgo --------
        riesgo_msg = ""
        umbral = rend_promedio_ruta.get(viaje)

        if umbral is not None and pred < umbral:
            riesgo_msg = (
                "La ruta tiene riesgo de subrendimiento con respecto al "
                "promedio histórico de la región."
            )
        # Si se quiere mostrar algo cuando NO hay riesgo:
        # elif umbral is not None:
        #     riesgo_msg = (
        #         "La ruta se encuentra dentro o por encima del rendimiento "
        #         "promedio histórico de la región."
        #     )

        return texto_pred, riesgo_msg, ""

    except Exception as e:
        return "", "", f"Error al predecir: {e}"

@app.callback(Output("page-content", "children"), Input("url", "pathname"))
def display_page(pathname):
    if pathname == "/panel":
        return layout_panel()
    elif pathname == "/ingresos":
        return layout_ingresos()
    elif pathname == "/unidades":
        return layout_unidades()
    elif pathname == "/demanda":
        return layout_demanda()
    elif pathname == "/rutas":
        return layout_rutas()
    else:
        return layout_inicio()


@app.callback(Output("clock", "children"), Input("clock-interval", "n_intervals"))
def update_clock(n):
    now = dt.datetime.now()
    return now.strftime("%d de %B de %Y — %I:%M %p")


@app.callback(Output("app-root", "className"), Input("theme-switch", "value"))
def toggle_theme(dark):
    return "dark-theme" if dark else "light-theme"


# -------------------------------------------------------------------
# RUN
# -------------------------------------------------------------------

if __name__ == "__main__":
    app.run(debug=True, port=8051)


c:\Users\gsant\Documents\TEC\Semestre7\WorkDash\.team-\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\gsant\Documents\TEC\Semestre7\WorkDash\.team-\.venv\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
